# Alignment in Bioinformatics

## The Needleman-Wunsch Algorithm

**Global sequence alignment** assumes two sequences are homologous (or share a common evolutionary ancestor) across their whole length. It tries to line up the whole thing, from tail to snout. This might be a reasonable assumption for two genes in the same gene family, for instance.

First, we import some modules which needed:

In [45]:
from IPython.display import HTML, display
import numpy as np
import pandas as pd

Then, we create class `NeedlemanWunsch` to implement the Needleman-Wunsch algorithm in python:

In [57]:
class NeedlemanWunsch:
    """ This is a dynamic programming algorithm for finding the optimal
    global alignment of two sequences.
    """
    def __init__(self, seq1, seq2, gap_penalty=-1, match_award=1, mismatch_penalty=-1):
        self.seq1, self.seq2 = seq1, seq2
        self.M, self.N = len(self.seq1)+1, len(self.seq2)+1
        self.gap_penalty = gap_penalty
        self.match_award = match_award
        self.mismatch_penalty = mismatch_penalty

    def pretty_table_from_array(self, data_array, row_labels, col_labels):
        """ Show an HTML table from a 2d numpy array """
        df = pd.DataFrame(data_array, index=row_labels, columns=col_labels)
        table_html = df.to_html()
        return HTML(table_html)

    def match_score(self, alpha, beta):
        """ Get the match score between two sequences """
        if alpha == beta:
            return self.match_award
        else:
            return self.mismatch_penalty

    def score_matrix(self):
        """ Get the score matrix and traceback matrix between two sequences """
        # Initiation
        score = np.zeros((self.M, self.N))
        trace = np.full([self.M, self.N], '-')
        # Define the arrows from different directions
        up_arrow = "\u2191"
        left_arrow = "\u2190"
        up_left_arrow = "\u2196"
        # Calculate
        # Fill out cell(0,0)
        score[0, 0] = 0
        trace[0, 0] = '-'
        # Fill out first column
        for i in range(1, self.M):
            score[i, 0] = self.gap_penalty * i
            trace[i, 0] = up_arrow
        # Fill out first row
        for j in range(1, self.N):
            score[0, j] = self.gap_penalty * j
            trace[0, j] = left_arrow
        # Fill out all other cell
        for i in range(1, self.M):
            for j in range(1, self.N):
                match = score[i-1, j-1] + self.match_score(self.seq1[i-1], self.seq2[j-1])
                delete = score[i-1, j] + self.gap_penalty
                insert = score[i, j-1] + self.gap_penalty
                score[i, j] = max(match, delete, insert)
                trace[i, j] = [up_left_arrow, left_arrow, up_arrow][np.argmax([match, delete, insert])]
        return score, trace

    def traceback(self):
        """ Traceback and calculate the alignment """
        # Get the score array from def(score_matrix)
        score, _ = self.score_matrix()
        # Create variables to store alignment
        align1 = ""
        align2 = ""
        # start from the bottom right cell
        i, j = self.M-1, self.N-1
        # Traceback
        while i > 0 and j > 0:
            # match or mismatch
            if score[i, j] == score[i-1, j-1] + self.match_score(self.seq1[i-1], self.seq2[j-1]):
                align1 += self.seq1[i-1]
                align2 += self.seq2[j-1]
                i -= 1
                j -= 1
            # delete
            elif score[i, j] == score[i-1, j] + self.gap_penalty:
                align1 += self.seq1[i-1]
                align2 += '-'
                i -= 1
            # insert
            elif score[i, j] == score[i, j-1] + self.gap_penalty:
                align1 += '-'
                align2 += self.seq2[j-1]
                j -= 1
        # Finish tracing
        while i > 0:
            align1 += self.seq1[i-1]
            align2 += '-'
            i -= 1
        while j > 0:
            align1 += '-'
            align2 += self.seq2[j-1]
            j -= 1
        # Reverse the alignment
        align1 = align1[::-1]
        align2 = align2[::-1]
        return align1, align2

    def display_alignment(self):
        """ Display the global alignment between two sequences """
        # Get the alignment result
        align1, align2 = self.traceback()
        # Get the map label index
        map_label = [" "] * len(align1)
        for i in range(len(align1)):
            if align1[i] == align2[i]:
                map_label[i] = "|"
            elif align1[i] == '-' or align2[i] == '-':
                continue
            else:
                map_label[i] = '.'

        map_label = ''.join(map_label)
        # Display the alignment
        print(align1 + "\n" + map_label + "\n" + align2)

    def display_array(self):
        """ Display the score array and trace array """
        # Get the score array and trace array
        score_array, traceback_array = self.score_matrix()
        # Display
        column_labels = [label for label in "-" + "".join(self.seq1)]
        row_labels = [label for label in "-" + "".join(self.seq2)]
        print('Score Matrix:')
        display(self.pretty_table_from_array(score_array, row_labels, column_labels))
        print('Traceback Matrix:')
        display(self.pretty_table_from_array(traceback_array, row_labels, column_labels))

Here's an example. Let's start with two random unaligned DNA sequences created by `numpy.random`:


In [58]:
np.random.seed(42)
x = np.random.choice(['A', 'T', 'G', 'C'], 8)
y = np.random.choice(['A', 'T', 'G', 'C'], 8)

We set different gap penalty [0, -1, -2] to test the Needleman-Wunsch algorithm:

In [59]:
nw0 = NeedlemanWunsch(x, y, gap_penalty=0)
nw1 = NeedlemanWunsch(x, y, gap_penalty=-1)
nw2 = NeedlemanWunsch(x, y, gap_penalty=-2)

Then, we check out the score matries and traceback matries, firstly.

In [60]:
nw1.display_array()

Score Matrix:


,-,G,C,A,G,G,C,A,A
-,0.0,-1.0,-2.0,-3.0,-4.0,-5.0,-6.0,-7.0,-8.0
G,-1.0,1.0,0.0,-1.0,-2.0,-3.0,-4.0,-5.0,-6.0
T,-2.0,0.0,0.0,-1.0,-2.0,-3.0,-4.0,-3.0,-4.0
G,-3.0,-1.0,-1.0,-1.0,-2.0,-3.0,-4.0,-4.0,-2.0
G,-4.0,-2.0,-2.0,0.0,0.0,-1.0,-2.0,-3.0,-3.0
G,-5.0,-3.0,-3.0,-1.0,1.0,1.0,0.0,-1.0,-2.0
G,-6.0,-4.0,-4.0,-2.0,0.0,0.0,0.0,1.0,0.0
C,-7.0,-5.0,-5.0,-3.0,-1.0,-1.0,-1.0,0.0,2.0
A,-8.0,-6.0,-6.0,-4.0,-2.0,-2.0,-2.0,-1.0,1.0


Traceback Matrix:


,-,G,C,A,G,G,C,A,A
-,-,←,←,←,←,←,←,←,←
G,↑,↖,↑,↖,↖,↖,↖,↑,↑
T,↑,←,↖,↖,↖,↖,↖,↖,↑
G,↑,←,↖,↖,↖,↖,↖,←,↖
G,↑,↖,↖,↖,↖,↖,↖,↑,←
G,↑,↖,↖,↖,↖,↖,↖,↑,↑
G,↑,←,↖,←,←,↖,↖,↖,↑
C,↑,←,↖,←,←,↖,↖,←,↖
A,↑,←,↖,←,←,↖,↖,←,↖


The score matrix shown above can now be read as a map that show better or worse paths through the alignment of seq1 and seq2, with higher numebrs representing better paths. 

There are a couple of rules for interpreting the matrix. They may or may not be intuitive at first, but if we follow them we can reconstruct all the optimal alignments from the matrix. 

- We start at the lower right. Needleman-Wunsch assumes a **global alignment**, meaning the whole sequence is aligned. Therefore we start at the end and work backward.
- Each horizontal step represents insertion of an indel('-') into the sequence of the rows.
- Each verical step represents insertion of an indel('-') into the top sequence.
- The optimal alignments are represented by the cells with the highest scores.

Then, we check out the alignment between seq1 and seq2 under different gap penalty:

In [65]:
print('gap penalty = 0:')
nw0.display_alignment() # gap penalty = 0
print("\n" + 'gap penalty = -1:')
nw1.display_alignment() # gap penalty = -1
print("\n" + 'gap penalty = -2:')
nw2.display_alignment() # gap penalty = -2


gap penalty = 0:
---GCAGGCAA
   |  ||| |
GTGG--GGC-A

gap penalty = -1:
G-CAGGCAA
| ..||| |
GTGGGGC-A

gap penalty = -2:
GCAGGCAA
|..||..|
GTGGGGCA


where, '|' represents match perfectly, '.' represents mismatch, and space represents indel between two sequences.

## The Smith-Waterman Algorithm

**Local sequence alignment** assumes that there are regions of homology within a bigger sequence, but also regions that do not share common ancestry. This might be a reasonable assumption if checking how many genes two species share. If the species aren't extremely closely related, we would expect that they would share certain genes, but also that there should be large portions of the genome that do not share detectable evolutionary ancestry between the two species (due to large insertions, deletions, inversions or rearrangements).

First, we import some modules needed:

In [25]:
from IPython.display import HTML, display
import numpy as np
import pandas as pd

Then, we create class `SmithWaterman` to implement the Smith-Waterman algorithm in python:

In [66]:
class SmithWaterman:
    """ This is a dynamic programming algorithm for finding the optimal
    local alignment of two sequences.
    """
    def __init__(self, seq1, seq2, gap_penalty=-1, match_award=1, mismatch_penalty=-1):
        self.seq1, self.seq2 = seq1, seq2
        self.M, self.N = len(self.seq1)+1, len(self.seq2)+1
        self.gap_penalty = gap_penalty
        self.match_award = match_award
        self.mismatch_penalty = mismatch_penalty
        self.max_score = -1
        self.max_index = (-1, -1)

    def pretty_table_from_array(self, data_array, row_labels, col_labels):
        """ Show an HTML table from a 2d numpy array """
        df = pd.DataFrame(data_array, index=row_labels, columns=col_labels)
        table_html = df.to_html()
        return HTML(table_html)

    def match_score(self, alpha, beta):
        """ Get the match score between two sequences """
        if alpha == beta:
            return self.match_award
        else:
            return self.mismatch_penalty

    def score_matrix(self):
        """ Get the score matrix between two sequences """
        # Initiation
        score = np.zeros((self.M, self.N))
        trace = np.full([self.M, self.N], '-')
        # Define the arrows from different directions
        up_arrow = "\u2191"
        left_arrow = "\u2190"
        up_left_arrow = "\u2196"
        # Calculate
        # Fill out cell(0,0)
        score[0, 0] = 0
        trace[0, 0] = '-'
        # Fill out first column
        for i in range(1, self.M):
            score[i, 0] = self.gap_penalty * i
            trace[i, 0] = up_arrow
        # Fill out first row
        for j in range(1, self.N):
            score[0, j] = self.gap_penalty * j
            trace[0, j] = left_arrow
        # Fill out all other cell
        for i in range(1, self.M):
            for j in range(1, self.N):
                match = score[i-1, j-1] + self.match_score(self.seq1[i-1], self.seq2[j-1])
                delete = score[i-1, j] + self.gap_penalty
                insert = score[i, j-1] + self.gap_penalty
                score[i, j] = max(0, match, delete, insert)
                trace[i, j] = [up_left_arrow, left_arrow, up_arrow][np.argmax([match, delete, insert])]
                # Get the best locally alignment
                if score[i, j] >= self.max_score:
                    self.max_index = (i, j)
                    self.max_score = score[i, j]
        return score, trace

    def traceback(self):
        """ Traceback and calculate the alignment """
        # Get the score matrix from def(score_matrix)
        score, _ = self.score_matrix()
        # Create variables to store alignment
        align1 = ""
        align2 = ""
        # start from the best score cell
        (i, j) = self.max_index
        # Traceback
        while i > 0 and j > 0:
            # set trace stop value
            if score[i, j] == 0:
                break
            # match or mismatch
            if score[i, j] == score[i-1, j-1] + self.match_score(self.seq1[i-1], self.seq2[j-1]):
                align1 += self.seq1[i-1]
                align2 += self.seq2[j-1]
                i -= 1
                j -= 1
            # delete
            elif score[i, j] == score[i-1, j] + self.gap_penalty:
                align1 += self.seq1[i-1]
                align2 += '-'
                i -= 1
            # insert
            elif score[i, j] == score[i, j-1] + self.gap_penalty:
                align1 += '-'
                align2 += self.seq2[j-1]
                j -= 1
        # Reverse the alignment
        align1 = align1[::-1]
        align2 = align2[::-1]
        return align1, align2

    def display_alignment(self):
        """ Display the global alignment between two sequences """
        # Get the alignment result
        align1, align2 = self.traceback()
        # Get the map label index
        map_label = [" "] * len(align1)
        for i in range(len(align1)):
            if align1[i] == align2[i]:
                map_label[i] = "|"
            elif align1[i] == '-' or align2[i] == '-':
                continue
            else:
                map_label[i] = '.'

        map_label = ''.join(map_label)
        # Display the alignment
        print(align1 + "\n" + map_label + "\n" + align2)

    def display_array(self):
        """ Display the score array and trace array """
        # Get the score array and trace array
        score_array, traceback_array = self.score_matrix()
        # Display
        column_labels = [label for label in "-" + "".join(self.seq1)]
        row_labels = [label for label in "-" + "".join(self.seq2)]
        print('Score Matrix:')
        display(self.pretty_table_from_array(score_array, row_labels, column_labels))
        print('Traceback Matrix')
        display(self.pretty_table_from_array(traceback_array, row_labels, column_labels))

Here's an example. Let's start with two random unaligned DNA sequences created by `numpy.random`:

In [67]:
np.random.seed(42)
x = np.random.choice(['A', 'T', 'G', 'C'], 8)
y = np.random.choice(['A', 'T', 'G', 'C'], 8)
sw = SmithWaterman(x, y, gap_penalty=-1)

Then, we check out the score matries and traceback matries, firstly.

In [69]:
sw.display_array()

Score Matrix:


,-,G,C,A,G,G,C,A,A
-,0.0,-1.0,-2.0,-3.0,-4.0,-5.0,-6.0,-7.0,-8.0
G,-1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
T,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
G,-3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
G,-4.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0
G,-5.0,0.0,0.0,1.0,2.0,2.0,2.0,1.0,0.0
G,-6.0,0.0,0.0,0.0,1.0,1.0,1.0,3.0,2.0
C,-7.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,4.0
A,-8.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0


Traceback Matrix


,-,G,C,A,G,G,C,A,A
-,-,←,←,←,←,←,←,←,←
G,↑,↖,↑,↖,↑,↑,↑,↑,↑
T,↑,←,↖,↖,↖,↖,↖,↖,↑
G,↑,←,↖,↖,↖,↖,↖,←,↖
G,↑,←,↖,↖,↖,↖,↖,↑,←
G,↑,←,↖,↖,↖,↖,↖,↑,←
G,↑,←,↖,←,←,↖,↖,↖,↑
C,↑,←,↖,↖,←,↖,↖,←,↖
A,↑,←,↖,↖,↖,↖,↖,←,↖


Then, we check out the alignment between seq1 and seq2 under different gap penalty:

In [71]:
sw.display_alignment()

GGCA
||||
GGCA
